In [1]:
!pip3 install minio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 4.0 MB/s eta 0:00:00


In [550]:
import requests
from minio import Minio
import os
import json
import io
import re

In [551]:
def put_json(minio_client, bucket_name, object_name, d):
    """
    jsonify a dict and write it as object to the bucket
    """
    # prepare data and corresponding data stream
    data = json.dumps(d).encode("utf-8")
    data_stream = io.BytesIO(data)
    data_stream.seek(0)

    # put data as object into the bucket
    minio_client.put_object(
        bucket_name=bucket_name,
        object_name=object_name,
        data=data_stream, length=len(data),
        content_type="application/json"
    )

In [552]:
client = Minio(endpoint='minio:9000',
               access_key=os.environ['MINIO_ROOT_USER'],
               secret_key=os.environ['MINIO_ROOT_PASSWORD'],
               secure = False)

In [521]:
pokemon_id = 1
while pokemon_id > 0:
    print(f'Fetching pokemon_id = {str(pokemon_id).zfill(5)}', end = '\r')
    url = f"https://pokeapi.co/api/v2/pokemon/{pokemon_id}/"
    response = requests.get(url)
    
    if response.text == 'Not Found':
        pokemon_id += 1
        break

    put_json(minio_client = client,
             bucket_name = 'datalake',
             object_name = f'/raw/pokemon/pokemon_{str(pokemon_id).zfill(5)}.json',
             d = response.json())
    pokemon_id += 1

In [537]:
evolution_id = 1
while evolution_id <= 600:
    print(f'Fetching evolution_id = {str(evolution_id).zfill(5)}', end = '\r')
    
    url = f"https://pokeapi.co/api/v2/evolution-chain/{evolution_id}/"
    response = requests.get(url)
    
    if response.text == 'Not Found':
        evolution_id += 1
        continue
        
    put_json(minio_client = client,
         bucket_name = 'datalake',
         object_name = f'/raw/evolution/evolution_{str(evolution_id).zfill(5)}.json',
         d = response.json())
    evolution_id += 1